In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
import pandas as pd

# 문제가 있는 줄을 건너뛰고 CSV 파일을 읽음
file_path = 'ChatBotData.csv'

chatbot_data = pd.read_csv(file_path, error_bad_lines=False)

# 데이터의 처음 몇 줄을 출력
chatbot_data.head()

#데이터 전처리하기

In [ ]:
MAX_SAMPLES = 50000
print(MAX_SAMPLES)

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()

    return sentence

In [ ]:
def load_conversations(questions, answers):
    inputs, outputs = [], []
    
    for question, answer in zip(questions, answers):
        inputs.append(preprocess_sentence(question))
        outputs.append(preprocess_sentence(answer))
        
    return inputs, outputs

In [ ]:
questions, answers = load_conversations(chatbot_data['Q'], chatbot_data['A'])
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

In [ ]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[21]))

#subword

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [ ]:
print('START_TOKEN의 번호 :', [tokenizer.vocab_size])
print('END_TOKEN의 번호 :', [tokenizer.vocab_size + 1])

In [ ]:
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

In [ ]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

In [ ]:
MAX_LENGTH = 40
print(MAX_LENGTH)

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :', (VOCAB_SIZE))
print('필터링 후의 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 샘플 개수: {}'.format(len(answers)))

#트랜스포머 모델 구성하기

1. 모델의 구조
트랜스포머는 인코더-디코더 구조로 되어 있으며, 인코더는 입력 문장을, 디코더는 출력 문장을 처리한다. 챗봇에서 트랜스포머를 사용할 경우, 보통 인코더가 사용자 입력을 이해하고, 디코더가 해당 입력을 바탕으로 응답을 생성하는 방식으로 두 개의 모듈이 서로 상호작용하며 정보를 처리한다.
반면 GPT는 트랜스포머의 디코더 부분만 사용하는 구조이다. GPT는 입력 문장의 맥락을 기반으로 다음 단어를 예측하는 언어 모델로, 자연스럽게 대화를 이어나가는 능력이 뛰어나다.

2. 훈련 방식
트랜스포머 기반 챗봇은 종종 입력과 출력 간의 매핑을 학습하는 방식으로 훈련된다. 이는 번역 작업에서 사용된 것처럼 입력을 처리한 후, 해당 입력에 맞는 적절한 출력 문장을 생성하는 방식이다. 이 모델은 보통 다수의 병렬적인 데이터로 학습되며, 인코더와 디코더 간의 복잡한 상호작용을 통해 문맥을 이해하게 된다.
반면 GPT는 사전 훈련과 미세 조정 방식을 따른다. 먼저, 대규모의 텍스트 데이터를 바탕으로 라벨링하지 않고 훈련한 후, 이를 바탕으로 특정 작업에 맞게 조정한다.

3. 실시간 대화 처리
트랜스포머는 입력과 출력을 명시적으로 처리하기 때문에 실시간 대화에서 입력을 처리하고 응답을 생성하기까지 시간이 다소 걸릴 수 있지만,
GPT는 실시간으로 다음 단어를 예측하면서 응답을 생성하는 방식으로 동작하기 때문에 대화의 흐름이 더욱 자연스럽고 빠르다